In [1]:
import re
import enum
import math
import random
import nltk
import csv
import numpy as np
import scipy as sp
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import semcor
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from random import randrange
from collections import deque
from statistics import mean

FUNZIONI DI PREPROCESSING

In [2]:
#il pre-processing consiste nella tokenizzazione, lemmatizzazione,
#rimozione della punteggiatura e delle stopwords di una sentence
def pre_processing(sentence):
    return set(remove_stopwords(tokenize_sentence(remove_punctuation(sentence))))

#rimuove le stowords da una lista di parole
def remove_stopwords(words_list):
    stopwords_list = get_stopwords()
    return [value.lower() for value in words_list if value.lower() not in stopwords_list]

#Tokenizza la frase in input e ne affettua anche la lemmatizzazione della sue parole
def tokenize_sentence(sentence):
    words_list = []
    lmtzr = WordNetLemmatizer()
    for tag in nltk.pos_tag(word_tokenize(sentence)):
        if (tag[1][:2] == "NN"):
            words_list.append(lmtzr.lemmatize(tag[0], pos = wn.NOUN))
        elif (tag[1][:2] == "VB"):
             words_list.append(lmtzr.lemmatize(tag[0], pos = wn.VERB))
        elif (tag[1][:2] == "RB"):
             words_list.append(lmtzr.lemmatize(tag[0], pos = wn.ADV))
        elif (tag[1][:2] == "JJ"):
             words_list.append(lmtzr.lemmatize(tag[0], pos = wn.ADJ))
    return words_list

#Restituisce la l'insieme di stopwords dal file delle stopwords
def get_stopwords():
    stopwords = open("stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords:
        stopwords_list.append(word.replace('\n', ''))
    stopwords.close()
    return stopwords_list

#Rimuove la punteggiatura da una sentence
#Restituisce la sentence senza punteggiature
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]','',sentence)

FUNZIONI UTILI PER L'ANALISI DEL TESTO

In [3]:
#Restituisce un sostantivo random presente nel dizionario associato ad una sentence
def get_random_noun(dictionary_tag):
    try:
        sentence_nouns = dictionary_tag['NN']
    except KeyError:
        return None #se non ci sono sostantivi
    noun =  random.choice(sentence_nouns)
    if len(wn.synsets(noun)) == 0:
        return None #se il sostantivo scelto non ha almeno un synset in wordnet
    return noun

#Restituisce una parola random presente nel dizionario associato ad una sentence
def get_random_word(dictionary_tag):
    keys = list(dictionary_tag.keys())
    if not keys:
        return None #se non ci sono parole "analizzabili" nel dizionario, aka se il dizionario è vuoto
    key = random.choice(keys)
    words_list = dictionary_tag[key]
    word = random.choice(words_list)
    if len(wn.synsets(word)) == 0:
        return None #se la parola scelta non ha almeno un synset in wordnet
    return word

#Verifica che una parola abbia un synset target associato
#Senza synset target associato è inutile confrontare l'output dell'algortimo di Lesk
def check_word_synset_target(word,sentence_sem):
    for w in sentence_sem:
        if (type(w) != list):
            if (w[0] == word):
                return True
    return False
    
#Crea un dizionario che ha come chiavi i tag presenti nella sentence e come valori liste di parole associate a quei tag
#e.g {NN: ['home','garden'], VB:['gone'], .....,}
#input: sentence_tag -> rappresenta le words della sentence con i relativi pos tag
#input: sentence_sem -> rappresenta le words della sentence con il relativo synset target associato
def get_dictionary_tag(sentence_tag,sentence_sem):
    dictionary_tag = dict()
    stopwords_list = get_stopwords()
    for word in sentence_tag:
         tag = word.label()
         word = " ".join(l for l in word)
         if check_word_synset_target(word, sentence_sem): #Non inserisco nel dizionario parole che non hanno un synset target
             w = word.lower()
             if w not in stopwords_list and tag: #Non inserisco nel dizionario stopwords o parole che non hanno un pos tag associato
                 if tag in dictionary_tag:
                     dictionary_tag[tag].append(word)
                 else:
                     dictionary_tag[tag] = [word]             
    return dictionary_tag

#Restituisce un indice randomico utile per l'ultimo esercizio
def get_random_index(index_evaluated, INDEXES_NUM):
    while True:
        index = randrange(INDEXES_NUM)
        if index not in index_evaluated:
            return index
    
#Restituisce il synset target per una parola in una sentence
def get_synset_target_for_word_in_sentence(noun,sentence):
     for word in sentence:
      if(type(word) != list):
          if (word[0] == noun):
              return str(word.label())

METRICHE DI SIMILARITA E ALGORITMI PER LA RICERCA

In [4]:
#Custom similarity functions
def wu_palmer_similarity(sense1, sense2):
    numerator = 2 * depth(lowest_common_subsumer(sense1,sense2))
    denominator = depth(sense1) + depth(sense2)
    return numerator/denominator

def leakcock_chodorow(sense1, sense2):
    numerator = (shortest_path(sense1, sense2) + 1)
    denominator = (2 * max_depth) + 1
    return -math.log(numerator/denominator)

def shortest_path_similarity(sense1, sense2):
    return ((2 * max_depth) - shortest_path(sense1, sense2))

#Wordnet similarity functions
def wn_wu_palmer_similarity(sense1, sense2):
    sim = sense1.wup_similarity(sense2)
    if sim: return sim
    else: return 0
    
def wn_leakcock_chodorow(sense1, sense2):
    try:
        sim = sense1.lch_similarity(sense2)
        if sim: return sim
        else: return 0
    except:
        return 0

#Restituisce la similarità tra due parole date in input
def similarity(word1, word2, similarity_function):
    max_sim = 0
    for s1 in wn.synsets(word1):
        for s2 in wn.synsets(word2):
           sim = similarity_function(s1,s2)
           if sim > max_sim:
               max_sim = sim
    return max_sim

#Restituisce la profondità di un senso in wordnet, cioè il massimo percorso dal senso alla radice        
def depth(sense):
    if not sense:
        return 0
    return max([len(path) for path in all_hypernym_paths(sense)])

#Restituisce tutti i percorsi di iperonimi di un senso di wordnet, la ricerca dei percosi viene effettuata in modo ricorsivo
def all_hypernym_paths(sense):
    paths = []
    hypernyms = sense.hypernyms()
    if len(hypernyms) == 0:
        paths = [[sense]]
    for hypernym in hypernyms:
        for ancestor_list in all_hypernym_paths(hypernym):
            ancestor_list.append(sense)
            paths.append(ancestor_list)
    return paths

#Massima profondità intesa come il percorso più lungo degli iperonimi di tutti i sensi presenti in wordnet
max_depth = max(max(len(hyp_path) for hyp_path in all_hypernym_paths(ss)) for ss in wn.all_synsets())

#Restituisce un dizionario di iperonimi del senso in input con la relativa profondità
def min_hypernyms_depth(sense):
    sense_depth_queue = deque([(sense, 0)]) #deque garantisce efficienza nell'inserire ed eliminare elementi sia a destra che a sinistra della coda
    sense_depth_dict = {}
    while sense_depth_queue:
        s, depth = sense_depth_queue.popleft()
        if s in sense_depth_dict:
            continue
        sense_depth_dict[s] = depth
        depth += 1
        sense_depth_queue.extend((hyperonym, depth) for hyperonym in s.hypernyms()) #li mette a destra della coda
    return sense_depth_dict

#restituisce il percorso più breve tra due sensi di wordnet
def shortest_path(sense1,sense2):
    if sense1 == sense2:
        return 0
    dict1 = min_hypernyms_depth(sense1)
    dict2 = min_hypernyms_depth(sense2)
    min_dist = (2 * max_depth) # distanza minima iniziale

    #per ogni senso del dizionario degli iperonimi di sense1 cerca di trovare il match con un senso del secondo dizionario.
    for ss, dist1 in dict1.items():
        dist2 = dict2.get(ss)
        if not dist2:
            dist2 = (2 * max_depth)
        min_dist = min(min_dist, dist1 + dist2) #somma dei due sensi matchati
    return (2 * max_depth) if math.isinf(min_dist) else min_dist

#Restituisce l'antenato comune più vicino di due sensi, cioè l'iperonimo comune più vicino ai due sensi. 
#SCELTA PROGETTUALE: Se ne ho più di uno prendo quello può essere il più profondo rispetto alla radice.
def lowest_common_subsumer(sense1, sense2):
    if sense1 == sense2: return sense1
    
    dict1 = min_hypernyms_depth(sense1)
    dict2 = min_hypernyms_depth(sense2)
    
    min_dist = (2 * max_depth)
    candidates = []
    for ss, dist1 in dict1.items():
        dist2 = dict2.get(ss) 
        if not dist2:
            dist2 = (2 * max_depth)
        else:
            if (dist1 + dist2) <= min_dist:
                min_dist = (dist1 + dist2)
                candidates.append(ss) 
    if candidates:
        deepest = candidates[0]
        for sense in candidates:
            if depth(sense) < depth(deepest):
                deepest = sense
        return deepest #restituisce il senso con profondità maggiore rispetto alla radice

#Classe enum con la quale specifichiamo il tipo di similarità utilizzata
class Similarity_Type(enum.Enum):
    wu_palmer_similarity = 1
    shortest_path = 2
    leakcock_chodorow = 3

Word sense disambiguation è un problema aperto dell'elaborazione del linguaggio naturale, che comprende il processo di identificazione del senso di una parola (cioè il significato) utilizzato in una determinata frase, quando la parola ha un numero di sensi distinti (polisemia).

In [5]:
#Algoritmo di Lesk
def lesk_algorithm(word, sentence, word_type):
    best_sense = wn.synsets(word)[0]
    max_overlap = 0
    max_signature = []
    print("Frase in cui compare la parola: ", sentence)
    context = pre_processing(sentence)
    
    #se il tipo di parola è un nome allora cerco solo tra i nomi in wordnet
    #altrimenti prendo tutti i synset associati a quel termine
    if word_type == 'NOUN':
        synsets = wn.synsets(word, pos=wn.NOUN)
    elif word_type == 'ALL':
        synsets = wn.synsets(word)
        
    for sense in synsets:
        signature = get_signature(sense)
        overlap = len(list(signature & context)) #overlap
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
            max_signature = signature
            
    print("Contesto della frase: ", context)
    print("Max Signature: ", max_signature) #signature del senso con più overlap con il contesto della sentence
    return best_sense

#Signature di un senso di WordNet
def get_signature(sense):
    signature = set()
    for word in pre_processing(sense.definition()): #tokenizzo la definizione del synset
        signature.add(word)
    for example in sense.examples(): #tokenizzo ogni esempio del synset
        for word in pre_processing(example):
            signature.add(word)
    return signature #la signature conterrà tutte le parole presenti nella definizione del senso e negli esempi

CONSEGNA 1: 

L'esercitazione consiste nell'implementare tre misure di similarità basate su WordNet. 

Per ciascuna di tali misure di similarità, calcolare gli indici di 
correlazione di Spearman and gli indici di correlazione di Pearson
fra i risultati ottenuti e quelli ‘target’ presenti nel file annotato.

In [6]:
def main():

    #scelta della metrica da utilizzare per il calcolo della similarità
    similarity_type = Similarity_Type.leakcock_chodorow
    #similarity_type = Similarity_Type.wu_palmer_similarity
    #similarity_type = Similarity_Type.shortest_path
    
    with open('WordSim353.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        assignments = []
        targets = []
        line_count = 0
        for row in csv_reader:
            if line_count > 0:
                print("Word1: ",row[0])
                print("Word2: ",row[1])
                sim = 0
                wn_sim = 0
                if similarity_type.value == 1:
                    sim = similarity(row[0], row[1], wu_palmer_similarity)
                    wn_sim = similarity(row[0], row[1], wn_wu_palmer_similarity)
                elif similarity_type.value == 2:
                    sim = similarity(row[0], row[1], shortest_path_similarity)
                elif similarity_type.value == 3:
                    sim = similarity(row[0], row[1], leakcock_chodorow)
                    wn_sim = similarity(row[0], row[1], wn_leakcock_chodorow)
                print("Type of similarity chosen: ", similarity_type.name)
                print("MyCustom Similarity: ",sim)
                print("WordNet Similarity: ",wn_sim)
                print("Human Similarity: ",row[2])
                assignments.append(sim)
                targets.append(float(row[2]))
                print("------------------------------")
            line_count = line_count + 1
            
    print("Pearson Correlation: ", np.corrcoef(assignments, targets))
    print("Spearman Correlation: ", sp.stats.spearmanr(assignments, targets))
main()

Word1:  love
Word2:  sex
Type of similarity chosen:  leakcock_chodorow
MyCustom Similarity:  2.327277705584417
WordNet Similarity:  2.9444389791664407
Human Similarity:  6.77
------------------------------
Word1:  tiger
Word2:  cat
Type of similarity chosen:  leakcock_chodorow
MyCustom Similarity:  2.327277705584417
WordNet Similarity:  2.9444389791664407
Human Similarity:  7.35
------------------------------
Word1:  tiger
Word2:  tiger
Type of similarity chosen:  leakcock_chodorow
MyCustom Similarity:  3.713572066704308
WordNet Similarity:  3.6375861597263857
Human Similarity:  10.00
------------------------------
Word1:  book
Word2:  paper
Type of similarity chosen:  leakcock_chodorow
MyCustom Similarity:  2.6149597780361984
WordNet Similarity:  2.538973871058276
Human Similarity:  7.46
------------------------------
Word1:  computer
Word2:  keyboard
Type of similarity chosen:  leakcock_chodorow
MyCustom Similarity:  2.327277705584417
WordNet Similarity:  2.2512917986064953
Human Sim

CONSEGNA 2.1: 

Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di 
WN) e disambiguare (almeno) un sostantivo per frase. 

Calcolare 
l’accuratezza del sistema implementato sulla base dei sensi annotati in 
SemCor.

In [7]:
tokenizer = RegexpTokenizer(r'\w+')

INDEXES_NUM = 1000 #è il limite, partendo da 0, di sentence che possono essere testate relative al database SemCor
RANGE = 50 #è il numero di sentence che effettivamente verranno testate

sentences_tag = semcor.tagged_sents() #lista di liste. Ogni lista rappresenta una sentence e ad ogni parola della sentence è associato un pos tag
sentences = semcor.sents() #lista di liste di parole. Ogni lista rappresenta una sentence
sentences_sem = semcor.tagged_sents(tag = "sem") #lista di liste. Ogni lista rappresenta una sentence e ad ogni parola della sentence è associato un synset gold

def main():
    evaluated = 0 #termini valutati
    checked = 0 #termini valutati correttamente
    index_evaluated = set() #insieme degli indici già valutati
    while(evaluated <= RANGE):
        while True: #fino a quando non trova una word che ha almeno un senso in wordnet   
            index = get_random_index(index_evaluated, INDEXES_NUM)
            word = get_random_noun(get_dictionary_tag(sentences_tag[index],sentences_sem[index]))
            if word:
                break
        sentence = sentences[index]
        print("Parola da disambiguare: ",word)
        #all'algoritmo di lesk viene dato in input la word e l'insieme dei termini che formano la frase, uniti sottoforma di stringa
        best_sense = str(lesk_algorithm(word, ' '.join(word for word in sentence), word_type='NOUN'))
        print("Senso attribuito dall'algoritmo di Lesk: ",best_sense)
        target_lemma = get_synset_target_for_word_in_sentence(word,sentences_sem[index])
        print("Senso Target: ",target_lemma)
        best_sense_lemma = best_sense[8:len(best_sense)-2]
        if target_lemma:
            if best_sense_lemma in target_lemma:
             checked= checked + 1
        evaluated += 1
        print("------------------------------")
    
    print("Termini valutati: ", evaluated)
    print("Termini valutati correttamente: ", checked)
    print("Accuratezza: ", checked/RANGE)
    
main()

Parola da disambiguare:  residents
Frase in cui compare la parola:  Nearly 18 per cent of West Berlin 's 2200000 residents are sixty-five or older , only 12.8 per cent are under fifteen .
Contesto della frase:  {'fifteen', 'west', 'cent', 'sixtyfive', 'berlin', 'resident'}
Max Signature:  {'hospital', 'patient', 'intern', 'special', 'training', 'clinical', 'physician', 'care', 'medical', 'receive', 'staff', 'supervision', 'hospitalized', 'resident', 'live'}
Senso attribuito dall'algoritmo di Lesk:  Synset('house_physician.n.01')
Senso Target:  Lemma('resident.n.01.resident')
------------------------------
Parola da disambiguare:  award
Frase in cui compare la parola:  The most valuable player award was split three ways , among Glen Mankowski , Gordon Hartweger and Tom Kieffer .
Contesto della frase:  {'tom', 'kieffer', 'glen', 'gordon', 'mankowski', 'valuable', 'hartweger', 'player', 'award', 'split'}
Max Signature:  {'distinction', 'signify', 'approval', 'tangible', 'symbol', 'award',

CONSEGNA 2.2:

Randomizzare la selezione delle 50 frasi e la selezione del termine da 
disambiguare, e restituire l’accuratezza media su (per esempio) 10 
esecuzioni del programma.

In [8]:
tokenizer = RegexpTokenizer(r'\w+')

EXECUTIONS = 10 #numero massimo di esecuzioni
INDEXES_NUM = 7000 #è il limite, partendo da 0, di sentence che possono essere testate relative al database SemCor
RANGE = 50 #è il numero di sentence che effettivamente verranno testate in ogni esecuzione        

sentences_tag = semcor.tagged_sents()
sentences = semcor.sents()
sentences_sem = semcor.tagged_sents(tag = "sem")

def main():
    accuracy_list = []
    for execution in range(1,EXECUTIONS + 1):
        evaluated = 0 #termini valutati
        checked = 0 #termini valutati correttamente
        index_evaluated = set() #insieme degli indici già valutati
        while(evaluated <= RANGE):
            while True: #fino a quando non trova una word che può essere presa in considerazione  
                index = get_random_index(index_evaluated, INDEXES_NUM)
                word = get_random_word(get_dictionary_tag(sentences_tag[index],sentences_sem[index]))
                if word:
                    break
            sentence = sentences[index]
            print("Parola da disambiguare: ",word)
            #all'algoritmo di lesk viene dato in input la word e l'insieme dei termini che formano la frase, uniti sottoforma di stringa
            best_sense = str(lesk_algorithm(word, ' '.join(word for word in sentence), word_type='ALL'))
            print("Senso attribuito dall'algoritmo di Lesk: ",best_sense)
            target_lemma = get_synset_target_for_word_in_sentence(word,sentences_sem[index])
            print("Senso Target: ",target_lemma)
            best_sense_lemma = best_sense[8:len(best_sense)-2]
            if target_lemma:
                if best_sense_lemma in target_lemma:
                 checked= checked + 1
            evaluated += 1
            print("------------------------------")
        
        print("Termini valutati: ", evaluated)
        print("Termini valutati correttamente: ", checked)
        accuracy = checked/RANGE
        print("Accuratezza: ", accuracy)
        accuracy_list.append(accuracy)
        
    print()
    print("Esecuzioni: ",EXECUTIONS)
    print("Lista delle accuratezze: ", accuracy_list)
    print("Accuratezza media: ",mean(accuracy_list))
    
main()

Parola da disambiguare:  releases
Frase in cui compare la parola:  All of the releases , however , are recorded at a gratifyingly high level , with resultant masking of any surface noise .
Contesto della frase:  {'gratifyingly', 'high', 'masking', 'surface', 'resultant', 'level', 'record', 'release', 'noise'}
Max Signature:  {'symphony', 'orchestra', 'release', 'sale', 'london', 'film', 'showing', 'record', 'public', 'issue', 'merchandise'}
Senso attribuito dall'algoritmo di Lesk:  Synset('release.n.01')
Senso Target:  Lemma('release.n.01.release')
------------------------------
Parola da disambiguare:  sees
Frase in cui compare la parola:  Not a circle , then , nor a straight line , but a spiral represents the shape of death as Irenaeus sees it ; for a spiral has motion as well as recurrence .
Contesto della frase:  {'death', 'shape', 'straight', 'recurrence', 'circle', 'represent', 'irenaeus', 'well', 'motion', 'spiral'}
Max Signature:  []
Senso attribuito dall'algoritmo di Lesk:  Sy